In [ ]:
import urllib
from urllib.request import urlopen
import feedparser
import numpy as np
import pandas as pd

# Base api query url
base_url = 'http://export.arxiv.org/api/query?';

# Search parameters
search_query = 'cat:cs.NI&sortBy=submittedDate&sortOrder=descending' # topic
start = 0                     
max_results = 100
test_df = pd.DataFrame()
arxivid=[]
dop = []
paper_title = []
all_authors = []
ref = []


while(start<5000):
    query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                         start,
                                                         max_results)
    start = start + 100
    # Opensearch metadata such as totalResults, startIndex, 
    # and itemsPerPage live in the opensearch namespase.
    # Some entry metadata lives in the arXiv namespace.
    # This is a hack to expose both of these namespaces in
    # feedparser v4.1
    feedparser._FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
    feedparser._FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'

    # perform a GET request using the base_url and query
    with urlopen(base_url+query) as url:
        response = url.read()

    print(base_url+query)
    # response = urllib.urlopen(base_url+query).read()

    # parse the response using feedparser
    feed = feedparser.parse(response)

    # print out feed information
    # print 'Feed title: %s' % feed.feed.title
    # print 'Feed last updated: %s' % feed.feed.updated

    # Run through each entry, and print out information
    for entry in feed.entries:
#         print ('e-print metadata')
#         print ('arxiv-id: %s' % entry.id.split('/abs/')[-1])
        arxivid.append(entry.id.split('/abs/')[-1])
        
#         print ('Published: %s' % entry.published)
        dop.append(entry.published)
        
#         print ('Title:  %s' % entry.title)
        paper_title.append(entry.title)

        # feedparser v4.1 only grabs the first author
        author_string = entry.author

        # grab the affiliation in <arxiv:affiliation> if present
        # - this will only grab the first affiliation encountered
        #   (the first affiliation for the first author)
        # Please email the list with a way to get all of this information!
#         try:
#             author_string += ' (%s)' % entry.arxiv_affiliation
#         except AttributeError:
#             pass

#         print ('Last Author:  %s' % author_string)

        # feedparser v5.0.1 correctly handles multiple authors, print them all
        try:
#             print ('Authors:  %s' % ', '.join(author.name for author in entry.authors))
            authors = []
            for author in entry.authors:
                authors.append(author.name)
#             print('Authors')
#             print(authors)
            all_authors.append(authors)
        except AttributeError:
            all_authors.append(authors)
            pass
        
        # get the links to the abs page and pdf for this e-print
    #     for link in entry.links:
    #         if link.rel == 'alternate':
    #             print 'abs page link: %s' % link.href
    #         elif link.title == 'pdf':
    #             print 'pdf link: %s' % link.href

        # The journal reference, comments and primary_category sections live under 
        # the arxiv namespace
        try:
            journal_ref = entry.arxiv_journal_ref
        except AttributeError:
            journal_ref = 'NA'
#         print ('Journal reference: %s' % journal_ref)
        ref.append(journal_ref)

#         try:
#             comment = entry.arxiv_comment
#         except AttributeError:
#             comment = 'No comment found'
#         print ('Comments: %s' % comment)

        # Since the <arxiv:primary_category> element has no data, only
        # attributes, feedparser does not store anything inside
        # entry.arxiv_primary_category
        # This is a dirty hack to get the primary_category, just take the
        # first element in entry.tags.  If anyone knows a better way to do
        # this, please email the list!
#         print ('Primary Category: %s' % entry.tags[0]['term'])

        # Lets get all the categories
        all_categories = [t['term'] for t in entry.tags]
#         print(all_categories)
#         print ('All Categories: %s' % (', ').join(all_categories))

        # The abstract is in the <summary> element
    #     print ('Abstract: %s' %  entry.summary)
    
    
    
test_df['arxiv_id'] = arxivid
test_df['title'] = paper_title
test_df['dop'] = dop
test_df['authors'] = all_authors
test_df['ref'] = ref

test_df.head()
# test_df.describe()

In [ ]:
test_df.to_csv('test_data_arxiv_networking.csv', index = False)
# stat.CO statistics
# cs.DC distributed
# cs.HC Human Computer Interaction
